# Objective: Obtain out of fold predictions on the entire training set using cross validation and then using a mean average precision IoU metric, that closely resembles the competition metric, to improve validation

In [169]:
import numpy as np
import pandas as pd

# Prepare out of fold training predictions for implementation of MAP IoU matching competition evaluation description

Load oof predictions from CNN segmentation CV kernel https://www.kaggle.com/cchadha/cnn-segmentation-cv-with-oof-preds-on-train-set/notebook

In [170]:
oof_preds0 = pd.read_csv('./root_dir/resnet50_Retina1.csv')

In [171]:
oof_preds0.head()

,patientId,PredictionString
0,28eaff00-5fb7-40ee-8206-9d2ff5b081e0,NaN
1,216ef465-167c-495f-b1ad-8218b65338ea,0.453 626 575 233 336 0.409 211 362 259 510
2,2bbcad9d-9dce-41f6-8642-d0fdce90abd5,NaN
3,273d9710-bda7-42f1-9068-ebe364b44128,NaN
4,2ce34f96-40b4-4d2a-b6a1-c7e21e6db8a1,0.353 266 489 225 276 0.183 627 373 222 355


Read in training labels

In [174]:
df = pd.read_csv('./input/stage_2_train_labels.csv')
df.head()

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


Parse bounding box labels into correct format for Mean Average Precision IoU metric

In [175]:
# make a bbox_target column using x, y, width and height.
df['bbox_target'] = (df['x'].astype(str) +
                    ' ' + 
                    df['y'].astype(str) +
                    ' ' +
                    df['width'].astype(str) +
                    ' ' +
                    df['height'].astype(str))

In [176]:
# 공백을 기준으로 나눔, ','는 기본으로 찍힘
df.loc[:, 'bbox_target'] = df.loc[:, 'bbox_target'].map(lambda x: x.split(' '))
df.head()

,patientId,x,y,width,height,Target,bbox_target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,"[264.0, 152.0, 213.0, 379.0]"


In [177]:
# 'patientId'를 기준으로 정렬 후, 동일 patientId가 있을 경우,
# bbox_target값 sum 하는데 string이라 옆으로 합쳐진다.
df = df.groupby(['patientId'], as_index = False)['bbox_target'].agg('sum')    
df.head()

,patientId,bbox_target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]"
1,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]"
2,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146..."
3,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296...."
4,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376...."


Merge labels and oof preds

In [178]:
#patientId를 기준으로 merge, 동일한 아이디가 없으면 삭제하기!
df = df.merge(oof_preds0, on = 'patientId', how = 'right')
df.head()

,patientId,bbox_target,PredictionString
0,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",NaN
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...",0.531 632 341 185 379 0.412 264 303 248 489
2,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....",0.853 77 219 314 440 0.842 548 262 309 426
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....",0.320 110 466 304 240 0.264 612 434 287 302
4,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",NaN


In [179]:
#3개의 파일을 merge 해서 PredictionString(x, y, default)로 뜸
df = df.fillna('') # none -> " "로 처리

In [180]:
df.head()

,patientId,bbox_target,PredictionString
0,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...",0.531 632 341 185 379 0.412 264 303 248 489
2,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....",0.853 77 219 314 440 0.842 548 262 309 426
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....",0.320 110 466 304 240 0.264 612 434 287 302
4,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",


Parse oof preds for MAP IoU

In [181]:
#필요없는 열 제거
#df = df.drop(['PredictionString','PredictionString_x', 'PredictionString_y'], axis=1)
df.loc[:, 'bbox_pred'] = (df.loc[:, 'PredictionString'])
df = df.drop(['PredictionString'], axis=1)
df.head(20)

,patientId,bbox_target,bbox_pred
0,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...",0.531 632 341 185 379 0.412 264 303 248 489
2,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....",0.853 77 219 314 440 0.842 548 262 309 426
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....",0.320 110 466 304 240 0.264 612 434 287 302
4,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",
5,001916b8-3d30-4935-a5d1-8eaddb1646cd,"[198.0, 375.0, 114.0, 206.0]",0.326 176 294 191 299
6,0022073f-cec8-42ec-ab5f-bc2314649235,"[575.0, 232.0, 246.0, 528.0, 161.0, 230.0, 223...",0.855 538 223 298 502 0.650 94 223 340 541
7,0022995a-45eb-4cfa-9a59-cd15f5196c64,"[nan, nan, nan, nan]",
8,0025d2de-bd78-4d36-9f72-e15a5e22ca82,"[nan, nan, nan, nan]",
9,00293de0-a530-41dc-9621-0b3def01d06d,"[nan, nan, nan, nan]",


Stripping whitespace from PredictionString column

In [182]:
# bbox_pred 값을 문자열로 변환 후 문자열의 양끝에서 공백, 탭, 개행문자 등을 제거
df = df.fillna('') # none -> " "로 처리
df.loc[:, 'bbox_pred'] = df.loc[:, 'bbox_pred'].str.strip()
df.tail(20)

,patientId,bbox_target,bbox_pred
980,c145df8e-9631-468d-af7f-5690c11c2c88,"[601.0, 265.0, 173.0, 383.0, 173.0, 370.0, 178...",0.504 163 355 216 281 0.196 552 352 276 422
981,c14d9ceb-019f-45f6-9299-281b58de57df,"[578.0, 447.0, 157.0, 258.0, 239.0, 268.0, 172...",0.635 174 337 246 477 0.381 526 436 252 390
982,c1592aae-c80d-4794-ab28-463905558534,"[nan, nan, nan, nan]",
983,c1628c47-5ba3-42dd-8df3-7ad3abd57ad0,"[622.0, 356.0, 149.0, 128.0]",0.416 572 205 271 543
984,c164b17b-aff8-484f-9d9e-dde2932d8df9,"[nan, nan, nan, nan]",0.251 152 553 237 258
985,c1718678-44af-407f-829a-fc65bc854094,"[nan, nan, nan, nan]",0.184 542 401 283 176 0.095 195 410 183 178
986,c18d1138-ba74-4af5-af21-bdd4d2c96bb5,"[646.0, 386.0, 141.0, 206.0, 295.0, 473.0, 133...",0.458 249 449 175 205 0.407 569 262 230 369
987,c196ce23-f37c-4ab3-a9ce-ea8ede90e09c,"[nan, nan, nan, nan]",
988,c19b8a3b-ab4e-4a73-8e13-ec0a84b6b6c7,"[nan, nan, nan, nan]",
989,c1c3ec5d-20ba-42f7-91f9-48032d97ffc9,"[nan, nan, nan, nan]",


In [183]:
# 공백을 기준으로 나눔, ','는 기본으로 찍힘
df.loc[:, 'bbox_pred'] = df.loc[:, 'bbox_pred'].map(lambda x: x.split(' '))
df.head()

,patientId,bbox_target,bbox_pred
0,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",[]
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...","[0.531, 632, 341, 185, 379, 0.412, 264, 303, 2..."
2,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....","[0.853, 77, 219, 314, 440, 0.842, 548, 262, 30..."
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....","[0.320, 110, 466, 304, 240, 0.264, 612, 434, 2..."
4,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",[]


In [184]:
def parse_scores(x):
    if len(x)!=1:
        scores = [x[k] for k in range(0,len(x),5)]
        for score in range(len(scores)):
            scores[score] = float(scores[score])
        return np.asarray(scores)
    #score 추출

In [185]:
df.loc[:, 'bbox_scores'] = df.loc[:, 'bbox_pred'].map(parse_scores)

In [186]:
df.tail()

,patientId,bbox_target,bbox_pred,bbox_scores
995,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,"[666.0, 418.0, 186.0, 223.0, 316.0, 504.0, 179...","[0.373, 319, 464, 189, 287, 0.208, 638, 351, 2...","[0.373, 0.208]"
996,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,"[609.0, 464.0, 240.0, 284.0, 185.0, 298.0, 228...","[0.606, 184, 210, 274, 510, 0.270, 598, 461, 2...","[0.606, 0.27]"
997,c1edf42b-5958-47ff-a1e7-4f23d99583ba,"[nan, nan, nan, nan]",[],None
998,c1f6b555-2eb1-4231-98f6-50a963976431,"[nan, nan, nan, nan]","[0.095, 243, 556, 183, 184]",[0.095]
999,c1f7889a-9ea9-4acb-b64c-b737c929599a,"[570.0, 393.0, 261.0, 345.0, 233.0, 424.0, 201...","[0.335, 234, 504, 220, 256, 0.125, 590, 458, 2...","[0.335, 0.125]"


In [187]:
def parse_bbox(x):
    if len(x)!=1:
        bbox = [(x[k]) for k in range(0,len(x)) if k%5 != 0] #int 삭제
        return np.asarray(bbox).reshape(int(len(bbox)/4),4)
    # bbox 추출

In [188]:
df.loc[:, 'bbox_preds'] = df.loc[:, 'bbox_pred'].map(parse_bbox)

In [189]:
df.tail()

,patientId,bbox_target,bbox_pred,bbox_scores,bbox_preds
995,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,"[666.0, 418.0, 186.0, 223.0, 316.0, 504.0, 179...","[0.373, 319, 464, 189, 287, 0.208, 638, 351, 2...","[0.373, 0.208]","[[319, 464, 189, 287], [638, 351, 224, 323]]"
996,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,"[609.0, 464.0, 240.0, 284.0, 185.0, 298.0, 228...","[0.606, 184, 210, 274, 510, 0.270, 598, 461, 2...","[0.606, 0.27]","[[184, 210, 274, 510], [598, 461, 268, 293]]"
997,c1edf42b-5958-47ff-a1e7-4f23d99583ba,"[nan, nan, nan, nan]",[],None,None
998,c1f6b555-2eb1-4231-98f6-50a963976431,"[nan, nan, nan, nan]","[0.095, 243, 556, 183, 184]",[0.095],"[[243, 556, 183, 184]]"
999,c1f7889a-9ea9-4acb-b64c-b737c929599a,"[570.0, 393.0, 261.0, 345.0, 233.0, 424.0, 201...","[0.335, 234, 504, 220, 256, 0.125, 590, 458, 2...","[0.335, 0.125]","[[234, 504, 220, 256], [590, 458, 237, 265]]"


In [190]:
#bbox_pred 추출 후 열 제거
df = df.drop(['bbox_pred'], axis=1)

In [191]:
df.tail()

,patientId,bbox_target,bbox_scores,bbox_preds
995,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,"[666.0, 418.0, 186.0, 223.0, 316.0, 504.0, 179...","[0.373, 0.208]","[[319, 464, 189, 287], [638, 351, 224, 323]]"
996,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,"[609.0, 464.0, 240.0, 284.0, 185.0, 298.0, 228...","[0.606, 0.27]","[[184, 210, 274, 510], [598, 461, 268, 293]]"
997,c1edf42b-5958-47ff-a1e7-4f23d99583ba,"[nan, nan, nan, nan]",None,None
998,c1f6b555-2eb1-4231-98f6-50a963976431,"[nan, nan, nan, nan]",[0.095],"[[243, 556, 183, 184]]"
999,c1f7889a-9ea9-4acb-b64c-b737c929599a,"[570.0, 393.0, 261.0, 345.0, 233.0, 424.0, 201...","[0.335, 0.125]","[[234, 504, 220, 256], [590, 458, 237, 265]]"


Edit NaN or None values to empty numpy arrays to fit MAP IoU metric implementation

In [192]:
df.loc[df['bbox_scores'].isnull(),['bbox_scores']] = df.loc[df['bbox_scores'].isnull(),'bbox_scores'].apply(lambda x: np.asarray([]))

In [193]:
#bbox_scores null 제거 후 출력
df.tail()

,patientId,bbox_target,bbox_scores,bbox_preds
995,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,"[666.0, 418.0, 186.0, 223.0, 316.0, 504.0, 179...","[0.373, 0.208]","[[319, 464, 189, 287], [638, 351, 224, 323]]"
996,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,"[609.0, 464.0, 240.0, 284.0, 185.0, 298.0, 228...","[0.606, 0.27]","[[184, 210, 274, 510], [598, 461, 268, 293]]"
997,c1edf42b-5958-47ff-a1e7-4f23d99583ba,"[nan, nan, nan, nan]",[],None
998,c1f6b555-2eb1-4231-98f6-50a963976431,"[nan, nan, nan, nan]",[0.095],"[[243, 556, 183, 184]]"
999,c1f7889a-9ea9-4acb-b64c-b737c929599a,"[570.0, 393.0, 261.0, 345.0, 233.0, 424.0, 201...","[0.335, 0.125]","[[234, 504, 220, 256], [590, 458, 237, 265]]"


In [194]:
df.loc[df['bbox_preds'].isnull(),['bbox_preds']] = df.loc[df['bbox_preds'].isnull(),'bbox_preds'].apply(lambda x: np.asarray([]))

In [195]:
#bbox_pred null 제거 후 출력
df.head()

,patientId,bbox_target,bbox_scores,bbox_preds
0,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",[],[]
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...","[0.531, 0.412]","[[632, 341, 185, 379], [264, 303, 248, 489]]"
2,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....","[0.853, 0.842]","[[77, 219, 314, 440], [548, 262, 309, 426]]"
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....","[0.32, 0.264]","[[110, 466, 304, 240], [612, 434, 287, 302]]"
4,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",[],[]


In [196]:
def parse_target_str(x):
    if x[0] != 'nan':
        bbox = np.asarray([int(float(x[k])) for k in range(0,len(x))])
        return bbox.reshape(int(len(bbox)/4),4)

In [197]:
df.loc[:,'bbox_target'] = df.loc[:,'bbox_target'].map(parse_target_str)
df.head()
#bbox_target값 4개씩 묶어서 재 배열

,patientId,bbox_target,bbox_scores,bbox_preds
0,000924cf-0f8d-42bd-9158-1af53881a557,None,[],[]
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[[316, 318, 170, 478], [660, 375, 146, 402]]","[0.531, 0.412]","[[632, 341, 185, 379], [264, 303, 248, 489]]"
2,000fe35a-2649-43d4-b027-e67796d412e0,"[[570, 282, 269, 409], [83, 227, 296, 438]]","[0.853, 0.842]","[[77, 219, 314, 440], [548, 262, 309, 426]]"
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[[66, 160, 373, 608], [552, 164, 376, 676]]","[0.32, 0.264]","[[110, 466, 304, 240], [612, 434, 287, 302]]"
4,0010f549-b242-4e94-87a8-57d79de215fc,None,[],[]


In [198]:
df.loc[df['bbox_target'].isnull(),['bbox_target']] = df.loc[df['bbox_target'].isnull(),'bbox_target'].apply(lambda x: np.asarray([]))

In [199]:
df.head()
# bbox_target null 제거 후 출력

,patientId,bbox_target,bbox_scores,bbox_preds
0,000924cf-0f8d-42bd-9158-1af53881a557,[],[],[]
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[[316, 318, 170, 478], [660, 375, 146, 402]]","[0.531, 0.412]","[[632, 341, 185, 379], [264, 303, 248, 489]]"
2,000fe35a-2649-43d4-b027-e67796d412e0,"[[570, 282, 269, 409], [83, 227, 296, 438]]","[0.853, 0.842]","[[77, 219, 314, 440], [548, 262, 309, 426]]"
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[[66, 160, 373, 608], [552, 164, 376, 676]]","[0.32, 0.264]","[[110, 466, 304, 240], [612, 434, 287, 302]]"
4,0010f549-b242-4e94-87a8-57d79de215fc,[],[],[]


# Find mean average precision IoU using implementation by chenyc15 https://www.kaggle.com/chenyc15/mean-average-precision-metric and edited herein

In [200]:
# helper function to calculate IoU
def iou(box1, box2):
    box1 = box1.astype(np.float)
    box2 = box2.astype(np.float)
    x11, y11, w1, h1 = box1
    x21, y21, w2, h2 = box2
    assert w1 * h1 > 0
    assert w2 * h2 > 0
    x12, y12 = x11 + w1, y11 + h1
    x22, y22 = x21 + w2, y21 + h2

    area1, area2 = w1 * h1, w2 * h2
    xi1, yi1, xi2, yi2 = max([x11, x21]), max([y11, y21]), min([x12, x22]), min([y12, y22])
    
    if xi2 <= xi1 or yi2 <= yi1:
        return 0
    else:
        intersect = (xi2-xi1) * (yi2-yi1)
        union = area1 + area2 - intersect
        return intersect / union

In [201]:
def map_iou(boxes_true, boxes_pred, scores, thresholds = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]):
    """
    Mean average precision at differnet intersection over union (IoU) threshold
    
    input:
        boxes_true: Mx4 numpy array of ground true bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
        boxes_pred: Nx4 numpy array of predicted bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
        scores:     length N numpy array of scores associated with predicted bboxes
        thresholds: IoU shresholds to evaluate mean average precision on
    output: 
        map: mean average precision of the image
    """
    
    # According to the introduction, images with no ground truth bboxes will not be 
    # included in the map score unless there is a false positive detection (?)
        
    # return None if both are empty, don't count the image in final evaluation (?)
    if len(boxes_true) == 0 and len(boxes_pred) == 0:
        return None
    elif len(boxes_true) == 0 and len(boxes_pred) > 0:
        return 0
    elif len(boxes_true) > 0 and len(boxes_pred) == 0:
        return 0
    elif len(boxes_true) > 0 and len(boxes_pred) > 0:
        assert boxes_true.shape[1] == 4 or boxes_pred.shape[1] == 4, "boxes should be 2D arrays with shape[1]=4"
        if len(boxes_pred):
            assert len(scores) == len(boxes_pred), "boxes_pred and scores should be same length"
            # sort boxes_pred by scores in decreasing order
            boxes_pred = boxes_pred[np.argsort(scores)[::-1], :]

        map_total = 0
        map_threshold = []

        # loop over thresholds
        for t in thresholds:
            matched_bt = set()
            tp, fn = 0, 0
            for i, bt in enumerate(boxes_true):
                matched = False
                for j, bp in enumerate(boxes_pred):
                    miou = iou(bt, bp)
                    if miou >= t and not matched and j not in matched_bt:
                        matched = True
                        tp += 1 # bt is matched for the first time, count as TP
                        matched_bt.add(j)
                if not matched:
                    fn += 1 # bt has no match, count as FN

            fp = len(boxes_pred) - len(matched_bt) # FP is the bp that not matched to any bt
            m = tp / (tp + fn + fp) #precision value
            map_threshold.append(m)
            map_total += m
    
        map_total=map_total / len(thresholds) #AP per image
        map_threshold.append(map_total)
    
    return map_threshold

In [202]:
#map_iou test, 상위 20개 행만 테스트
for row in range(20):
    print(map_iou(df['bbox_target'][row], df['bbox_preds'][row], df['bbox_scores'][row]))

None
[1.0, 1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 0.0, 0.0, 0.6666666666666666]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
None
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125]
[1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.6666666666666665]
None
None
None
0
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 0.0, 0.0, 0.20833333333333331]
0
None
None
None
0
[1.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 0.0, 0.0, 0.29166666666666663]
None
0


In [203]:
#non이 아닐때만 map_iou 계산
map_scores = [
    x for x in [map_iou(df['bbox_target'][row], df['bbox_preds'][row], df['bbox_scores'][row]) for row in range(len(df))] if x is not None]

In [204]:
# null -> [0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(len(map_scores)):
    if map_scores[i] == 0 :
        del map_scores[i]
        map_scores.insert(i,[0, 0, 0, 0, 0, 0, 0, 0, 0])

In [205]:
new_list = list(map(list, zip(*map_scores))) # reverse, [i][j]->[j][i]

In [206]:
thres_list = []
for i in range(len(new_list)):
    thres_list.append(np.mean(new_list[i])) # AP per thresholds

In [207]:
thres_list #임계치별 AP (0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, (kaggle mAP))

[0.22886297376093295,
 0.18886227960571983,
 0.15788560322088016,
 0.13581146744412048,
 0.11115160349854228,
 0.08600583090379009,
 0.05830903790087463,
 0.039358600583090375,
 0.12578092461474386]

In [208]:
for i in np.mean(thres_list)

0.12578092461474383